# **Recurrent Neural Network and LSTM from Scratch**

In [ ]:
!git clone https://github.com/arindam96/deep-learning-specialization-coursera.git

Cloning into 'deep-learning-specialization-coursera'...
remote: Enumerating objects: 766, done.
remote: Counting objects: 100% (181/181), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 766 (delta 32), reused 121 (delta 19), pack-reused 585 (from 1)
Receiving objects: 100% (766/766), 172.44 MiB | 18.72 MiB/s, done.
Resolving deltas: 100% (113/113), done.
Updating files: 100% (558/558), done.


In [ ]:
%cd /content/deep-learning-specialization-coursera/Course 5 - Sequence Models/Week 1 - Recurrent Neural Networks/2. Programming Assignment Building your Recurrent Neural Network - Step by Step

/content/deep-learning-specialization-coursera/Course 5 - Sequence Models/Week 1 - Recurrent Neural Networks/2. Programming Assignment Building your Recurrent Neural Network - Step by Step


In [ ]:
!ls

Building_a_Recurrent_Neural_Network_Step_by_Step.ipynb	__pycache__
generateTestCases.py					rnn_utils.py
images							test_utils.py
public_tests.py						utils.py


# Building Neural Network

We'll implement Recurrent Neural Network, or RNN and LSTM in Numpy!



## Table of Content

- [Packages](#0)
- [1 - Forward Propagation for Basic Recurrent Neural Network](#1)
    - [1.1 - RNN cells](#1-1)
    - [2.2 - RNN Forward Pass](#1-2)
- [2 - Long Sort-Term Memory (LSTM) Network](#2)
    - [2.1 - LSTM Cell](#2-1)
    - [2.2 - Forward Pass for LSTM](#2-2)
    

<a name='0'></a>
## Packages

In [12]:
import numpy as np
from rnn_utils import *

<a name='1'></a>
## 1 - Forward Propagation for the Basic Recurrent Neural Network


<a name='1-1'></a>
### 1.1 - RNN Cell


### rnn_cell_forward

In [5]:
def rnn_cell_forward(xt, a_prev, parameters):
  Wax=parameters["Wax"]
  Waa=parameters["Waa"]
  Wya=parameters["Wya"]
  ba=parameters["ba"]
  by=parameters["by"]

  # Compute next activation state
  a_next=np.tanh(np.dot(Waa, a_prev)+np.dot(Wax, xt)+ba)

  # Compute output of the current cell
  yt_pred=softmax(np.dot(Wya, xt)+by)

  # Store values that we need to store in cache for backpropagation
  cache=(a_next, a_prev, xt, parameters)

  return a_next, yt_pred, cache

<a name='1-2'></a>
### 1.2 - RNN Forward Pass



### rnn_forward

In [6]:
def rnn_forward(x, a0, parameters):

  # Initialize "caches" which will contain all the list of caches
  cache=[]

  # Retreive dimensions from shapes of x and paramerters["Wya"]
  n_x, m, T_x=x.shape
  n_y, n_a=parameters["Wya"].shape

  # Initialize "a" and "y_pred" with zeros
  a=np.zeros((n_a, m, T_x))
  y_pred=np.zeros((n_y, m, T_x))

  # Initialize a_next
  a_next=a0

  for t in range(T_x):
    # Update net hidden state, compute prediction, get the cache
    a_next, yt_pred, cache=rnn_cell_forward(x[:, :, t], a_next, parameters)
    # Save the value of the new "nexr" hidden state
    a[:, :, t]=a_next
    # Save the value of prediction in y
    y_pred[:, :, t]=yt_pred
    # Append "cache" to "caches"
    caches.append(cache)

  caches=(caches, x)

  return a, y_pred, caches


***We've successfully built the forward propagation of a recurrent neural network from scratch!***

<a name='2'></a>
## 2 - Long Short-Term Memory(LSTM) Network



<a name='2.1'></a>
### LSTM Cell

### lstm_cell_forward

In [7]:
def lstm_cell_forward(xt, a_prev, c_prev, parameters):

  # Retrieve parameters from "parameters"
  Wf=parameters["Wf"]
  bf=parameters["bf"]
  Wi=parameters["Wi"]
  bi=parameters["bi"]
  Wc=parameters["Wc"]
  bc=parameters["bc"]
  Wo=parameters["Wo"]
  bo=parameters["bo"]
  Wy=parameters["Wy"]
  by=parameters["by"]

  # Retrieve dimensions from shapes of xt and Wy
  n_x, m=xt.shape
  n_y, n_a=Wy.shape

  # Concatenate a_prev and x_t
  concat=np.concatenate((a_prev, xt), axis=0)

  # Compute values of ft, it, ot, cct, c_next, a_next
  ft=sigmoid(np.dot(Wf, concat)+bf)
  it=sigmoid(np.dot(Wi, concat)+bi)
  cct=np.tanh(np.dot(Wc, concat)+bc)
  c_next=ft*c_prev+it*cct
  ot=sigmoid(np.dot(Wo, concat)+bo)
  a_next=ot*np.tanh(c_next)

  # Compute prediction of the LSTM Cell
  yt_pred=softmax(np.dot(Wy, a_next)+by)

  # Store values for backward propagation in cache
  cache=(a_next, c_next, a_prev, c_prev, ft, it, cct, ot, xt, parameters)

  return a_next, c_next, yt_pred, cache

<a name='2-2'></a>
### 2.2 - Forward Pass for LSTM

### lstm_forward

In [11]:
def lstm_forward(x, a0, parameters):

  # Initialize "caches", which will track list of all caches
  caches=[]

  Wy=parameters["Wy"]

  # Retrieve dimensions from shapes of x and parameters["Wy"]
  n_x, m, T_x=x.shape
  n_y, n_a=Wy.shape

  # Initialize "a", "c" and "y" with zeros
  a=np.zeros((n_a, m, T_x))
  c=np.zeros((n_a, m, T_x))
  y=np.zeros((n_y, m, T_x))

  # Loop over all time steps
  for t in range(T_x):
    # Get the 2D slice'xt' from 3D input 'x' at time step 't'
    xt=x[:, :, t]

    # Update next hidden state, next memory state, compute the prediction, get the cache
    a_next, c_next, yt, cache=lstm_cell_forward(xt, a_next, c_next, parameters)
    # Save the value of the new "next" hidden state in a (≈1 line)
    a[:,:,t] = a_next
    # Save the value of the next cell state (≈1 line)
    c[:,:,t]  = c_next
    # Save the value of the prediction in y (≈1 line)
    y[:,:,t] = yt
    # Append the cache into caches (≈1 line)
    caches.append(cache)

  # store values needed for backward propagation in cache
  caches = (caches, x)

  return a, y, c, caches

***We'v Sucessfully implemented forward passes for both the Basic RNN and LSTM!***